In [3]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode
from pyspark.sql.functions import split
from pyspark.sql.types import *
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import LogisticRegressionModel


In [4]:
# Creamos la sesion de nuestro spark
spark = SparkSession \
    .builder \
    .appName("StructuredNetworkWordCount") \
    .getOrCreate()

In [ ]:
schema = StructType([StructField("Time",DoubleType(),True),StructField("V1",DoubleType(),True),StructField("V2",DoubleType(),True),StructField("V3",DoubleType(),True),
StructField("V4",DoubleType(),True),StructField("V5",DoubleType(),True),StructField("V6",DoubleType(),True),StructField("V7",DoubleType(),True),StructField("V8",DoubleType(),True),
StructField("V9",DoubleType(),True),StructField("V10",DoubleType(),True),StructField("V11",DoubleType(),True),StructField("V12",DoubleType(),True),StructField("V13",DoubleType(),True),
StructField("V14",DoubleType(),True),StructField("V15",DoubleType(),True),StructField("V16",DoubleType(),True),StructField("V17",DoubleType(),True),StructField("V18",DoubleType(),True),
StructField("V19",DoubleType(),True),StructField("V20",DoubleType(),True),StructField("V21",DoubleType(),True),StructField("V22",DoubleType(),True),StructField("V23",DoubleType(),True),
StructField("V24",DoubleType(),True),StructField("V25",DoubleType(),True),StructField("V26",DoubleType(),True),StructField("V27",DoubleType(),True),StructField("V28",DoubleType(),True),
StructField("features",DoubleType(),True),StructField("Amount",DoubleType(),True)])

lines = spark \
    .readStream \
    .format("kafka")\
    .option("kafka.bootstrap.server","localhost:9092")
    .option("subscribe","credit_topic")\
    .load()\
    .selectExpr("CAST(value AS STRING)")\
    .select(from_json(col("value"),schema).alias("parsed_value"))\
    .select("parsed_value.*")


In [ ]:
model = LogisticRegressionModel.load("modelLR")

In [ ]:
predictions = model.transform(lines)
y_pred = predictions.select('Class')



In [ ]:
# Generate running word count
query = y_pred \
    .writeStream \
    .format("parquet")\
    .outputMode("append") \
    .option("path", "hive/warehouse/credit_card_fraud")\
    .option("checkpointLocation", "hive/checkpoint/credit_card_fraud")\
    .start()

query.awaitTermination()